In [1]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 
import zipfile
from keras.layers import Dense, Reshape
import matplotlib.pyplot as plt
import math
from keras.losses import *
from keras.models import Sequential, Model
from keras.layers import Dense, Reshape
from keras.layers import LSTM, Flatten, Input, Embedding, concatenate, Concatenate, average, maximum, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from keras import optimizers
from keras import backend as K
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [2]:
def balanceFraudDataframe(df, p): #p is desired fraud/non-fraud ratio
    dfFraud = df.loc[df['isFraud']==1,:]
    x = len(dfFraud)
    n = len(df)
    numOfTimesToResample = int((p * n - x)/((1-p) * x))
    #print(numOfTimesToResample)
    dfBalanced = df
    for _ in range(numOfTimesToResample):
        dfBalanced = dfBalanced.append(dfFraud)   
    return dfBalanced

In [3]:
def findPercentFraud(df):
    return len(df.loc[df.isFraud == 0, :])/len(df)

In [4]:
df = pd.read_pickle("data/dataframe.pd")

In [8]:
scaler = MinMaxScaler(feature_range=(-1, 1))
scaler.fit(df.drop('isFraud', axis=1))

MinMaxScaler(copy=True, feature_range=(-1, 1))

In [9]:
choose = np.random.rand(len(df))
dfTrain = df.loc[choose >= .2, :]
dfTest = df.loc[choose < .2, :]

In [10]:
assert len(dfTrain) + len(dfTest) == len(df)

In [11]:
print(findPercentFraud(dfTrain))
dfTrain = balanceFraudDataframe(dfTrain, .25)
print(findPercentFraud(dfTrain))

0.9651152457885672
0.7545397703400568


In [12]:
print(findPercentFraud(dfTest))

0.9645880337240665


In [13]:
x_train = scaler.transform(dfTrain.drop(["isFraud"], axis=1))
y_train= np.array(dfTrain['isFraud'])

x_test = scaler.transform(dfTest.drop(["isFraud"], axis=1))
y_test = np.array(dfTest['isFraud'])

In [14]:
from sklearn.utils import class_weight

inputs = Input(shape=(312,))
# We stack a deep densely-connected network on top
x = Dense(400, activation='relu', use_bias=True)(inputs)
x = Dense(400,  activation='relu', use_bias=True)(x)
x = Dense(400, activation='relu', use_bias=True)(x)
x = Dense(400, activation='relu', use_bias=True)(x)
x = Dense(400, activation='relu', use_bias=True)(x)
# And finally we add the main logistic regression layer
output = Dense(1 , name='ANN_output')(x)

adam = optimizers.Adam(lr=0.01)
model = Model(inputs=inputs, outputs = output)


weights = class_weight.compute_class_weight('balanced',
                                            np.unique(y_train),
                                            y_train)

model.compile(loss="mean_squared_error", optimizer=adam, metrics=['accuracy'])
model.fit(x_train, y_train, epochs=50, batch_size=1024, verbose=2 )#, class_weight= weights)
print()


Epoch 1/50


KeyboardInterrupt: 

In [ ]:
trainPredict = model.predict(x_train)
testPredict = model.predict(x_test)
# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train, trainPredict))
#trainScore = numpy.mean(numpy.abs((y_train - trainPredict))/y_train)

print('Train Score: %.2f MAPE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test, testPredict))
#testScore =  numpy.mean(numpy.abs((y_test - testPredict))/y_test)

print('Test Score: %.2f MAPE' % (testScore))
keras.metrics.binary_accuracy(y_true, y_pred)

